In [10]:
import inspect
from typing import (
    Any,
    Callable,
    Coroutine,
    Dict,
    ForwardRef,
    List,
    Mapping,
    Optional,
    Sequence,
    Tuple,
    Type,
    Union,
    cast,
)
from typing_extensions import Annotated, get_args, get_origin

if True:
    from pydantic._internal._typing_extra import eval_type_lenient as evaluate_forwardref
else:
    from pydantic.typing import evaluate_forwardref as evaluate_forwardref


def get_typed_annotation(annotation: Any, globalns: Dict[str, Any]) -> Any:
    if isinstance(annotation, str):
        annotation = ForwardRef(annotation)
        annotation = evaluate_forwardref(annotation, globalns, globalns)
    return annotation

def get_typed_signature(call: Callable[..., Any]) -> inspect.Signature:
    signature = inspect.signature(call)
    globalns = getattr(call, "__globals__", {})
    typed_params = [
        inspect.Parameter(
            name=param.name,
            kind=param.kind,
            default=param.default,
            annotation=get_typed_annotation(param.annotation, globalns),
        )
        for param in signature.parameters.values()
    ]
    typed_signature = inspect.Signature(typed_params)
    return typed_signature

def get_typed_return_annotation(call: Callable[..., Any]) -> Any:
    signature = inspect.signature(call)
    annotation = signature.return_annotation

    if annotation is inspect.Signature.empty:
        return None

    globalns = getattr(call, "__globals__", {})
    return get_typed_annotation(annotation, globalns)


In [51]:
def f(a: Annotated[str, "param a"], b: int, c: "float", d=2):
    pass

In [52]:
inspect.signature(f).parameters

mappingproxy({'a': <Parameter "a: typing_extensions.Annotated[str, 'param a']">,
              'b': <Parameter "b: int">,
              'c': <Parameter "c: 'float'">,
              'd': <Parameter "d=2">})

In [53]:
[k for k, v in get_typed_signature(f).parameters.items() if v.annotation is inspect.Signature.empty]

['d']

In [59]:
typed_signature = get_typed_signature(f)
param_annotations = {k: v.annotation for k, v in typed_signature.parameters.items()}
return_annotation = get_typed_return_annotation(f)
missing_annotations = [k for k, v in param_annotations.items() if v is inspect.Signature.empty]
print(f"{typed_signature=}")
print(f"{param_annotations=}")
print(f"{return_annotation=}")
print(f"{missing_annotations=}")


typed_signature=<Signature (a: typing_extensions.Annotated[str, 'param a'], b: int, c: float, d=2)>
param_annotations={'a': typing_extensions.Annotated[str, 'param a'], 'b': <class 'int'>, 'c': <class 'float'>, 'd': <class 'inspect._empty'>}
return_annotation=None
missing_annotations=['d']


In [65]:
def get_required_params(typed_signature: inspect.Signature) -> List[str]:
    """Get the required parameters of a function

    Args:
        signature: The signature of the function as returned by inspect.signature

    Returns:
        A list of the required parameters of the function
    """
    return [k for k, v in typed_signature.parameters.items() if v.default == inspect.Signature.empty]


required = get_required_params(typed_signature)
required

['a', 'b', 'c']

In [66]:
def get_parameters(required: List[str], param_annotations: Dict[str, Union[Annotated[Type, str], Type]]) -> "Parameters":
    """Get the parameters of a function as defined by the OpenAI API

    Args:
        required: The required parameters of the function
        hints: The type hints of the function as returned by typing.get_type_hints

    Returns:
        A Pydantic model for the parameters of the function
    """
    return dict(
        properties={k: f"get_parameter_json_schema({k}, {v})" for k, v in param_annotations.items() if k != "return"}, required=required
    )

get_parameters(required, param_annotations)

{'properties': {'a': "get_parameter_json_schema(a, typing_extensions.Annotated[str, 'param a'])",
  'b': "get_parameter_json_schema(b, <class 'int'>)",
  'c': "get_parameter_json_schema(c, <class 'float'>)",
  'd': "get_parameter_json_schema(d, <class 'inspect._empty'>)"},
 'required': ['a', 'b', 'c']}

In [55]:
[k for k, v in param_annotations.items() if v.default == inspect.Signature.empty]

AttributeError: type object 'str' has no attribute 'default'

In [40]:
missing = [f"'{k}'" for k, v in param_annotations.items() if v is inspect.Signature.empty]
missing

["'d'"]

In [27]:
{k: v.annotation for k, v in get_typed_signature(f).parameters.items()}

{'a': typing_extensions.Annotated[str, 'param a'],
 'b': int,
 'c': float,
 'd': inspect._empty}

In [24]:
{k: f"get_parameter_json_schema({k}, {v})" for k, v in get_typed_signature(f).parameters.items()}

{'a': "get_parameter_json_schema(a, a: typing_extensions.Annotated[str, 'param a'])",
 'b': 'get_parameter_json_schema(b, b: int)',
 'c': 'get_parameter_json_schema(c, c: float)'}

In [68]:
get_args(Union[int, float])

(int, float)